## Geth

`geth` is the entry point into the Ethereum network (main-, test- or private net), capable of running as a full node (default) archive node (retaining all historical state) or a light node (retrieving data live). It can be used by other processes as a gateway into the Ethereum network via JSON RPC endpoints exposed on top of HTTP, WebSocket and/or IPC transports.

**Public Network**

This is the real PoW Ethereum network, where you use real ether. Use this netowrk only once you are confident of your operations and tested properly your smart contracts on a private or test network.

To run a full node on the main ehtereum network:

```bash
geth console
```

The command will do two things:

- Start geth in fast sync mode (default, can be changed with the `--syncmode` flag), causing it to download more data in exchange for avoiding processing the entire history of the Ethereum network, which is very CPU intensive.
- Start up Geth's built-in interactive JavaScript console, (via the trailing console subcommand) through which you can invoke all official web3 methods as well as Geth's own management APIs. This too is optional and if you leave it out you can always attach to an already running Geth instance with `geth attach`.


**Test Network**

Transitioning towards developers, if you'd like to play around with creating Ethereum contracts, you almost certainly would like to do that without any real money involved until you get the hang of the entire system. In other words, instead of attaching to the main network, you want to join the test network with your node, which is fully equivalent to the main network, but with play-Ether only.

Geth can connect to two different test networks:

- *Ropsten*: Best reproduces the current production environment, i.e. system and network conditions on the live Ethereum mainnet, because it's PoW net. Ether can be mined, or requested from a faucet. A downside is that it's not immune to spam attacks (due to PoW).  `geth --testnet` or `geth --networkid 3`
- *Rinkeby*: PoA (Proof of Authority) testnet, immune to spam attacks (Ether supply is controlled by trusted parties). The downside is that it doesn't fully reproduce the current production environment as it uses PoA. Ether has to be requested from a faucet. `geth --rinkeby` or `geth --networkid 4`

**Private network**

An even more convenient way to test and play around with smart contracts is to use a private network. Indeed you can avoid downloading the entire chains of test networks (in the order of GB of size), and spin up a network in a matter of seconds. Setting up the first configuration of a private network can be a little bit more complicated than connecting to an existing one, though.

**Light node**

TODO

Light nodes do not verify every block or transaction and may not have a copy of the current blockchain state. They rely on full nodes to provide them with missing details (or simply lack particular functionality). The advantage of light nodes is that they can get up and running much more quickly, can run on more computationally/memory constrained devices, and don’t eat up nearly as much storage. On the downside, there is an element of trust in other nodes (it varies based on client and probabilistic methods/heuristics can be used to reduce risk).

You can start a geth light node with:

```bash
geth --light
```

## Setup a private network

#### Single node network

Setting up a single node private network is a matter of running a single command.

```bash
geth --dev --rpc --ipcpath ~/Library/Ethereum/geth.ipc --datadir ~/Library/Ethereum/mytestnet
```

The `--dev` flag tells `geth` to run the node in private mode (does not connect to any public network). The `--rpc` flag will enable the HTTP-RPC server, so that you can call the node's functions with an appropriate RPC library (e.g. Web3.py). The `--datadir` flag tells `geth` where to place the blockchain data. Finally `--ipcpath` indicates the locations of the ipc socket. 

> IPC is short for inter-process communication and is no standard file in your installation or environment but rather a domain socket. After you installed geth the ipc socket is not automatically created and its also not considerable a permanent resource.
>
> Mind that the `geth.ipc` socket only exists as long as geth is running. You can control the available apis with the `--ipcapi` flag. You can disable IPC with `--ipcdisable`.

#### Multiple nodes network

**Choose a network ID**

Since connections between nodes are valid only if peers have identical protocol version and network ID, you can effectively isolate your network by setting either of these to a non default value. You can use the `--networkid` command line option for this. The main network has id 1 (the default). So if you supply your own custom network ID which is different than the main network your nodes will not connect to other nodes and form a private network.

**Custom Genesis Block**

Every blockchain starts with a genesis block, so we need to define a custom one to bootstrap our private chain. All the nodes need to be aware of this genesis block and agree upon it.

```json
{
    "config": {
        "chainId": 12345,
        "homesteadBlock": 0,
        "eip155Block": 0,
        "eip158Block": 0
    },
    "difficulty": "200000000",
    "gasLimit": "2100000",
    "alloc": {
        "7df9a875a174b3bc565e6424a0050ebc1b2d1d82": { "balance": "300000" },
        "f41c74c9ae680c1aa78f42e5647a62f353b7bdde": { "balance": "400000" }
    }
}
```

TO initialize a chain that uses this block, run the following command:

```bash
geth --datadir path/to/data/folder init genesis.json
```

You can set the data folder to any custom data folder, just like the single node private chain we used before.

After initialization, you can run nodes pointing to this data directory specifying the network id:

```bash
geth --datafir path/to/data/folder --networkid 12345
```

**Bootnode - Enable peer discovery**

In a peer-to-peer netowrk, new nodes need some way to discover and connect to other nodes during the bootstrap phase. We can initialize a dedicated *bootstrap* node for this purpose:

```bash
bootnode --genkey=boot.key
bootnode --nodekey=boot.key
```

Once online, the bootstrap node will display an URL that other nodes can use to connect to it.

> NOTE: Make sure to replace the displayed IP address information (most probably [::]) with your externally accessible IP to get the actual enode URL.

**Start up new members**

With the bootnode operational and externally reachable, start every subsequent Geth node pointed to the bootnode for peer discovery via the `--bootnodes` flag. It will probably also be desirable to keep the data directory of your private network separated, so do also specify a custom `--datadir` flag.

```bash
geth --datadir path/to/data/folder --networkid 12345 --bootnodes <bootnode-enode-url-from-above>
```

**Running a private miner**

TODO

Explain the concept of the DAG: https://ethereum.stackexchange.com/questions/1993/what-actually-is-a-dag

#### Back up the `/keystore`!

Each account’s private keys are encrypted and stored in the /keystore folder of the datadir. These should be backed up off of the hard drive in case of removal- whether accidental or malicious.

- To restore an account after /keystore was removed, simply paste the backed-up files back to /keystore under the data directory
- If trying to just clean up the blockchain, can do so with geth upgradedb or geth removedb – these will not affect the keystore

## Py-Geth

Py-Geth is a Python wrapper around running `geth` as a subprocess. The only requiremnet is the `geth` executable.

**Installation**

```bash
pip install py-geth
```

**Basic Usage**

You can easily connect to the main net buy running:

In [ ]:
from geth import LiveGethProcess

geth = LiveGethProcess()
geth.start()
# ... interact with the node using an rpc library
geth.close()

To launch a private network you need define a custom and, if needed, some overrides to default settings. By default the `DevGethProcess` sets up test chains in the default `datadir` used by `geth`. If you would like to change the location for these test chains, you can specify an alternative `base_dir`

In [ ]:
from geth import LoggingMixin, DevGethProcess

# Give a custon name to your private chain
CHAIN_NAME = 'dev_chain'

When testing it can be nice to see the logging output produced by the geth process. `py-geth` provides a mixin class that can be used to log the stdout and stderr output to a logfile.

All logs will be written to logfiles in `./logs/` in the current directory.

You can easily attach a separate shell to the log files:

```bash
tail -f ./logs/geth-<timestamp>-stderr.log
tail -f ./logs/geth-<timestamp>-stdout.log
```

In [ ]:
class MyLoggingGeth(LoggingMixin, DevGethProcess):
    pass

In [ ]:
# You can override some of the default parameters that DevGethProcess sets.
overrides = {
    'mine': False,  # ----> Check how to disable mining. Throws error with miner_threads.
    'miner_threads': 0
}
geth = MyLoggingGeth(chain_name=CHAIN_NAME, base_dir="./{}/testing".format(CHAIN_NAME), overrides=overrides)

# Start the process
geth.start()

TODO: It seems that DevGethProcess does not allow to disable mining (e.g., the generation of the DAG used for Ethash). Investigate this.

In [22]:
# You can get a lot of information about the state of the process
def get_node_status():
    print("RPC Ready: {}".format(geth.is_rpc_ready))
    print("IPC Ready: {}".format(geth.is_ipc_ready))
    print("DAG Generated Ready: {}".format(geth.is_dag_generated))
    print("IS MINING: {}".format(geth.is_mining))

In [22]:
# To stop the process:
geth.stop()

## References

- https://github.com/ethereum/go-ethereum#running-a-private-miner
- https://github.com/ethereum/go-ethereum/wiki/Private-network
- Geth Command Line Options: https://github.com/ethereum/go-ethereum/wiki/Command-Line-Options